The tutorials use PyTorch. You will need to load the following dependencies.

In [1]:
import random

import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import skimage.transform
import torch
import torch.nn as nn
import torch.utils.data
import torchvision
from IPython import display
from torchvision import datasets, transforms

# Tutorial 2a: Convolutional Neural Network (CNN)

The code below may be helpful in visualizing PyTorch tensors as images.

In [2]:
%matplotlib inline


def show(img):
    """Show PyTorch tensor img as an image in matplotlib."""
    npimg = img.cpu().detach().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation="nearest")
    plt.grid(False)
    plt.gca().axis("off")


def display_thumb(img):
    display.display(transforms.Resize(128)(img))

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Load MNIST and define train/test functions as before. Please make sure you read the code carefully and understand what it is doing.

In [4]:
# Load the training and test dataset.
mnist_train = datasets.MNIST(
    "/tmp/mnist", train=True, download=True, transform=transforms.ToTensor()
)
mnist_test = datasets.MNIST(
    "/tmp/mnist", train=False, download=True, transform=transforms.ToTensor()
)

# Size of the batches the data loader will produce.
batch_size = 64

# This creates the dataloaders.
train_loader = torch.utils.data.DataLoader(
    mnist_train, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    mnist_test, batch_size=batch_size, shuffle=False
)

100%|██████████| 9912422/9912422 [00:00<00:00, 129778502.38it/s]


Extracting /tmp/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 159389070.82it/s]


Extracting /tmp/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 107311300.71it/s]

Extracting /tmp/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/mnist/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 3804779.06it/s]

Extracting /tmp/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/mnist/MNIST/raw



In [5]:
def train(model, criterion, data_loader, optimizer, num_epochs):
    """Simple training loop for a PyTorch model."""

    # Make sure model is in training mode.
    model.train()

    # Move model to the device (CPU or GPU).
    model.to(device)

    # Exponential moving average of the loss.
    ema_loss = None

    # Loop over epochs.
    for epoch in range(num_epochs):

        # Loop over data.
        for batch_idx, (data, target) in enumerate(data_loader):

            # Forward pass.
            output = model(data.to(device))
            loss = criterion(output.to(device), target.to(device))

            # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # NOTE: It is important to call .item() on the loss before summing.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01

        # Print out progress the end of epoch.
        print(
            "Train Epoch: {} \tLoss: {:.6f}".format(epoch, ema_loss),
        )


def test(model, data_loader):
    """Measures the accuracy of a model on a data set."""
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0

    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():

        # Loop over test data.
        for data, target in data_loader:

            # Forward pass.
            output = model(data.to(device))

            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)

            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    percent = 100.0 * correct / len(data_loader.dataset)
    print(f"Accuracy: {correct} / {len(data_loader.dataset)} ({percent:.0f}%)")
    return percent

In the last tutorial, you implemented a naive convolution. In this section you will implement your own version of forward pass of nn.Conv2d without using any of PyTorch's (or numpy's) pre-defined convolutional functions.

In [15]:
def conv_forward_naive(x, w, b, conv_param):
    """
    A naive Python implementation of a convolutional layer.
    The input consists of N data points, each with C channels, height H and
    width W. We convolve each input with F different filters, where each filter
    spans all C channels and has height HH and width WW.
    Input:
    - x: Input data of shape (N, C, H, W)
    - w: Filter weights of shape (F, C, HH, WW)
    - b: Biases, of shape (F,)
    - conv_param: A dictionary with the following keys:
      - 'stride': The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.
      - 'pad': The number of pixels that will be used to zero-pad the input.

    During padding, 'pad' zeros should be placed symmetrically (i.e., equally on both sides)
    along the height and width axes of the input. Be careful not to modfiy the original
    input x directly.
    Returns an array.
    - out: Output data, of shape (N, F, H', W') where H' and W' are given by
      H' = 1 + (H + 2 * pad - HH) / stride
      W' = 1 + (W + 2 * pad - WW) / stride
    """
    out = None

    N, C, H, W = x.shape
    num_filters, _, filter_height, filter_width = w.shape
    stride, pad = conv_param["stride"], conv_param["pad"]
    
    # Check dimensions.
    assert (W + 2 * pad - filter_width) % stride == 0, "width does not work"
    assert (H + 2 * pad - filter_height) % stride == 0, "height does not work"

    ###########################################################################
    # TODO: Implement the forward pass of a convolutional layer without using #
    #       nn.Conv2D or other implementations of convolutions. Instead, use  #
    #       standard for- and while-loops to iterate over the tensors.        #
    #                                                                         #
    # Hint: you can use the function torch.nn.functional.pad for padding.     #
    ###########################################################################
    H_prime = 1 + (H + 2 * pad - filter_height) // stride
    W_prime = 1 + (W + 2 * pad - filter_width) // stride

    out= torch.zeros((N, num_filters, H_prime,W_prime))
    

    for n in range(N):
      for f in range(num_filters):
        sum = 0
        for c in range(C):
          out_chanel= torch.zeros((H_prime,W_prime))

          for i in range(H_prime):
            for j in range(W_prime):
              x_pad = nn.functional.pad(x[n,c],(pad, pad, pad, pad))
              
              out_chanel[i,j] = torch.sum(x_pad[i*stride:i*stride + filter_height, j*stride:j*stride + filter_width]*w[f, c])
              #print(out_chanel)
          sum = sum + out_chanel
        out[n, f] = sum  + b[f] 
        print(out.shape)
    return out

You can test your implementation by running the following testing code:

In [16]:
# Make convolution module.
w_shape = (3, 3, 4, 4)
w = torch.linspace(-0.2, 0.3, steps=torch.prod(torch.tensor(w_shape))).reshape(w_shape)
b = torch.linspace(-0.1, 0.2, steps=3)
print(b)

# Compute output of module and compare against reference values.
x_shape = (2, 3, 4, 4)
x = torch.linspace(-0.1, 0.5, steps=torch.prod(torch.tensor(x_shape))).reshape(x_shape)
out = conv_forward_naive(x, w, b, {"stride": 2, "pad": 1})

correct_out = torch.tensor(
    [
        [
            [[-0.08759809, -0.10987781], [-0.18387192, -0.2109216]],
            [[0.21027089, 0.21661097], [0.22847626, 0.23004637]],
            [[0.50813986, 0.54309974], [0.64082444, 0.67101435]],
        ],
        [
            [[-0.98053589, -1.03143541], [-1.19128892, -1.24695841]],
            [[0.69108355, 0.66880383], [0.59480972, 0.56776003]],
            [[2.36270298, 2.36904306], [2.38090835, 2.38247847]],
        ],
    ]
)

# Compare your output to ours; difference should be around e-8
print("Testing conv_forward_naive")
rel_error = ((out - correct_out) / (out + correct_out + 1e-6)).mean()
print("difference: ", rel_error)
if abs(rel_error) < 1e-6:
    print("Nice work! Your implementation of a convolution layer works correctly.")
else:
    print(
        "Something is wrong. The output was expected to be {} but it was {}".format(
            correct_out, out
        )
    )

tensor([-0.1000,  0.0500,  0.2000])
torch.Size([2, 3, 2, 2])
torch.Size([2, 3, 2, 2])
torch.Size([2, 3, 2, 2])
torch.Size([2, 3, 2, 2])
torch.Size([2, 3, 2, 2])
torch.Size([2, 3, 2, 2])
Testing conv_forward_naive
difference:  tensor(3.4319e-08)
Nice work! Your implementation of a convolution layer works correctly.



We will now replace the logistic regressor from the last tutorial by a small convolutional network with two convolutional layers and a linear layer, and ReLU activations in between the layers. Implement the model and use the same functions as before to train and test the convolutional network.

In [8]:
from torch.nn.modules import MaxPool2d
from torch.nn.functional import conv2d
class ConvolutionalNetwork(nn.Module):
    """Simple convolutional network."""

    def __init__(self, image_side_size, num_classes, in_channels=1):
        super(ConvolutionalNetwork, self).__init__()
        self.in_channels = 1
        self.num_classes = num_classes
        self.image_side_size = image_side_size

        # Fill these in:
        ##########################################################################
        # TODO: Implement a convulutional and a linear part.                     #
        # Hint: see forward() to understand how they should work together.       #
        ##########################################################################
        self.conv_network = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride =1),
        
           # second layers
            nn.Conv2d(in_channels = 8, out_channels=8, kernel_size= 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride=1)

            
        )
        conv_output_size = self.get_conv_network_output()
        self.linear = nn.Linear(conv_output_size, num_classes)
       
       
    def get_conv_network_output(self, ):
      dummy_x = torch.zeros(1, self.in_channels, self.image_side_size, self.image_side_size)
      conv_output = self.conv_network(dummy_x)
      conv_output_size = torch.prod(torch.tensor(conv_output.shape))
      return conv_output_size
      #self.linear = nn.Linear(conv_output_size, num_classes)

    def forward(self, x):
        x = self.conv_network(x)
        x = self.linear(x.view(x.size(0), -1))
        return x


# Create and train convolutional network.
# The accuracy should be around 96%.
conv_model = ConvolutionalNetwork(28, 10)
###########################################################################
# TODO: Create criterion and optimize here.                               #
###########################################################################
import torch.optim as optim 
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(conv_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-3)

train(conv_model, criterion, train_loader, optimizer, num_epochs=5)
test(conv_model, test_loader)

KeyboardInterrupt: ignored

Inspect the filters in the first layer of the trained convolutional network. What do they look like? Why?

In [ ]:
first_conv = list(conv_model.conv_network.children())[0]
show(
    torchvision.utils.make_grid(
        first_conv.weight,
        normalize=True,
        nrow=8,
    )
)